### Créer un composant
Un composant se compose de trois parties :

- Métadonnées : inclut le nom, la version, etc. du composant.
- Interface : inclut les paramètres d’entrée attendus (comme un jeu de données ou un hyperparamètre) et la sortie attendue (comme des métriques et des artefacts).
- Commande, code et environnement : spécifie comment exécuter le code.

Pour créer un composant, vous avez besoin de deux fichiers :

Script qui contient le workflow que vous souhaitez exécuter.
Fichier YAML définissant les métadonnées, l’interface et la commande, le code et l’environnement du composant.
Vous pouvez créer le fichier YAML ou utiliser la fonction command_component() en tant qu’élément décoratif pour créer le fichier YAML.

Nous allons maintenant nous concentrer sur la création d’un fichier YAML pour créer un composant. Vous pouvez également en savoir plus en consultant le guide pratique pour créer des composants à l’aide de command_component(): https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-create-your-first-pipeline#build-a-pipeline-with-a-python-script



Vous pouvez par exemple disposer d’un script Python prep.py qui prépare les données en supprimant les valeurs manquantes et en normalisant les données : 

```python
# import libraries
import argparse
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

# setup arg parser
parser = argparse.ArgumentParser()

# add arguments
parser.add_argument("--input_data", dest='input_data',
                    type=str)
parser.add_argument("--output_data", dest='output_data',
                    type=str)

# parse args
args = parser.parse_args()

# read the data
df = pd.read_csv(args.input_data)

# remove missing values
df = df.dropna()

# normalize the data    
scaler = MinMaxScaler()
num_cols = ['feature1','feature2','feature3','feature4']
df[num_cols] = scaler.fit_transform(df[num_cols])

# save the data as a csv
output_df = df.to_csv(
    (Path(args.output_data) / "prepped-data.csv"), 
    index = False
)
```

Pour créer un composant pour le script prep.py, vous avez besoin d’un fichier YAML prep.yml :
```yaml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare training data
version: 1
type: command
inputs:
  input_data: 
    type: uri_file
outputs:
  output_data:
    type: uri_file
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python prep.py 
  --input_data ${{inputs.input_data}}
  --output_data ${{outputs.output_data}}
```

Notez que le fichier YAML fait référence au script prep.py qui est stocké dans le dossier src. Vous pouvez charger le composant avec le code suivant :
    
```python
from azure.ai.ml import load_component
parent_dir = ""

loaded_component_prep = load_component(source=parent_dir + "./prep.yml")
```


Une fois que vous avez chargé le composant, vous pouvez l’utiliser dans un pipeline ou l’inscrire.

### Inscrire un component
Pour utiliser des composants dans un pipeline, vous avez besoin du script et du fichier YAML. Pour que les composants soient accessibles aux autres utilisateurs de l’espace de travail, vous pouvez également inscrire des composants dans l’espace de travail Azure Machine Learning.

Vous pouvez inscrire un composant avec le code suivant :
```bash
prep = ml_client.components.create_or_update(prepare_data_component)
```



## Créer un pipeline

Un pipeline Azure Machine Learning est défini dans un fichier YAML. Le fichier YAML comprend le nom du travail de pipeline, les entrées, les sorties et les paramètres.

Vous pouvez créer le fichier YAML ou utiliser la fonction @pipeline() pour créer le fichier YAML.

Si vous souhaitez par exemple créer un pipeline qui prépare d’abord les données, puis effectue l’apprentissage du modèle, vous pouvez utiliser le code suivant :
```python
from azure.ai.ml.dsl import pipeline

@pipeline()
def pipeline_function_name(pipeline_job_input):
    prep_data = loaded_component_prep(input_data=pipeline_job_input)
    train_model = loaded_component_train(training_data=prep_data.outputs.output_data)

    return {
        "pipeline_job_transformed_data": prep_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
    }
```

Pour transmettre une ressource de données inscrite comme entrée du travail de pipeline, vous pouvez appeler la fonction que vous avez créée avec la ressource de données comme entrée :
```python
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

pipeline_job = pipeline_function_name(
    Input(type=AssetTypes.URI_FILE, 
    path="azureml:data:1"
))
```